In [1]:
import pandas as pd
from itertools import combinations

# Load the input file
data_file_path = 'C:/Users/saiki/OneDrive/Desktop/Sampledata.csv'
data = pd.read_csv(data_file_path, encoding='ISO-8859-1')

# Define the output file path
output_file_path = 'C:/Users/saiki/OneDrive/Desktop/normalization_output1.txt'
with open(output_file_path, 'w') as output_file:

    # Prompt for primary keys
    primary_keys = input("Enter primary key(s), separated by commas: ").split(',')
    primary_keys = [key.strip() for key in primary_keys]

    # Prompt for functional dependencies
    functional_dependencies = {}
    print("Provide Functional Dependencies in the format 'A,B -> C,D' (type 'done' when finished):")
    while True:
        fd_input = input()
        if fd_input.lower() == 'done':
            break
        try:
            determinant, dependent = fd_input.split("->")
            determinant_attrs = [attr.strip() for attr in determinant.split(',')]
            dependent_attrs = [attr.strip() for attr in dependent.split(',')]
            functional_dependencies[tuple(determinant_attrs)] = dependent_attrs
            output_file.write(f"Functional Dependency: {determinant} -> {dependent}\n")
        except ValueError:
            print("Invalid format. Please use 'A,B -> C,D'.")

    # Get the highest normal form to achieve
    highest_form = input("Select the highest normal form to achieve (1: 1NF, 2: 2NF, 3: 3NF, B: BCNF, 4: 4NF, 5: 5NF): ")
    if highest_form in ["1", "2", "3", "4", "5"]:
        highest_form = int(highest_form)

    # Prompt for multi-valued dependencies if the target form is 4NF or higher
    multi_valued_dependencies = {}
    if highest_form >= 4:
        print("Provide Multi-Valued Dependencies in the format 'A ->> B' (type 'done' when finished):")
        while True:
            mvd_input = input()
            if mvd_input.lower() == 'done':
                break
            try:
                determinant, dependent = mvd_input.split("->>")
                determinant = determinant.strip()
                dependent = dependent.strip()
                if determinant in multi_valued_dependencies:
                    multi_valued_dependencies[determinant].append(dependent)
                else:
                    multi_valued_dependencies[determinant] = [dependent]
                output_file.write(f"Multi-Valued Dependency: {determinant} ->> {dependent}\n")
            except ValueError:
                print("Invalid format. Please use 'A ->> B'.")

    # Prompt for join dependencies if the target form is 5NF
    join_dependencies = []
    if highest_form == 5:
        print("Provide Join Dependencies in the format 'A,B' (type 'done' when finished):")
        while True:
            jd_input = input()
            if jd_input.lower() == 'done':
                break
            jd_attrs = [attr.strip() for attr in jd_input.split(',')]
            join_dependencies.append(jd_attrs)
            output_file.write(f"Join Dependency: {', '.join(jd_attrs)}\n")

    # Function to ensure 1NF by handling non-atomic columns
    def apply_1nf(df):
        non_atomic_cols = []
        for col in df.columns:
            if df[col].apply(lambda x: isinstance(x, list)).any():
                non_atomic_cols.append(col)
                df = df.explode(col)
        return df, non_atomic_cols

    # Function to check 2NF
    def validate_2nf(df, primary_keys, dependencies):
        non_prime_columns = [col for col in df.columns if col not in primary_keys]
        for det, dep in dependencies.items():
            if set(det).issubset(primary_keys) and any(attr in non_prime_columns for attr in dep):
                return False
        return True

    # Function to check 3NF
    def validate_3nf(df, primary_keys, dependencies):
        for det, dep in dependencies.items():
            if set(det) != set(primary_keys) and not set(dep).issubset(primary_keys):
                return False
        return True

    # Function to check BCNF
    def validate_bcnf(df, primary_keys, dependencies):
        for det, dep in dependencies.items():
            if not set(det).issubset(primary_keys):
                return False
        return True

    # Function to apply 4NF using multi-valued dependencies
    def apply_4nf(df, mv_dependencies):
        decomposed_tables = []
        remaining_columns = list(df.columns)

        for det, dep_list in mv_dependencies.items():
            for dep in dep_list:
                decomposed_table = df[[det, dep]].drop_duplicates()
                decomposed_tables.append((decomposed_table, [det, dep]))  # Track primary key for decomposed table
                if dep in remaining_columns:
                    remaining_columns.remove(dep)
                output_file.write(f"4NF Decomposition: Created table with {det} ->> {dep}\n")

        main_table = df[remaining_columns].drop_duplicates()
        return [(main_table, primary_keys)] + decomposed_tables

    # Function to apply 5NF using join dependencies
    def apply_5nf(df, join_deps):
        decomposed_tables = []

        for jd in join_deps:
            if set(jd).issubset(df.columns):
                decomposed_table = df[jd].drop_duplicates()
                decomposed_tables.append((decomposed_table, jd))
                output_file.write(f"5NF Decomposition: Created table for JD ({', '.join(jd)})\n")

        if not decomposed_tables:
            output_file.write("No further 5NF decomposition required.\n")
        return [(df, primary_keys)] + decomposed_tables

    # SQL Query Generator and Schema Representation
    def generate_sql_schema(table_name, df, primary_keys):
        sql_query = f"CREATE TABLE {table_name} (\n"
        schema_info = f"Table: {table_name}\nAttributes:\n"

        for col, dtype in zip(df.columns, df.dtypes):
            col_type = 'INT' if 'int' in str(dtype) else 'VARCHAR(255)'
            sql_query += f"  {col} {col_type},\n"
            schema_info += f"  - {col} ({col_type})\n"

        primary_key_str = ", ".join(primary_keys)
        sql_query += f"  PRIMARY KEY ({primary_key_str})\n);"
        schema_info += f"Primary Key(s): {primary_key_str}\n\n"

        print(sql_query)
        output_file.write(sql_query + "\n\n" + schema_info)

    # Main Normalization Process
    def normalize_data(df, primary_keys, fds, mvds, highest_form, jds=[]):
        df, non_atomic_cols = apply_1nf(df)
        output_file.write("Applied 1NF\n" if not non_atomic_cols else f"1NF: Converted non-atomic columns {non_atomic_cols}\n")
        generate_sql_schema('Table_1NF', df, primary_keys)

        current_nf = 1
        if highest_form >= 2 and validate_2nf(df, primary_keys, fds):
            output_file.write("2NF validation successful.\n")
            current_nf = 2
        else:
            output_file.write("2NF decomposition needed.\n")

        if highest_form >= 3 and validate_3nf(df, primary_keys, fds):
            output_file.write("3NF validation successful.\n")
            current_nf = 3
        else:
            output_file.write("3NF decomposition needed.\n")

        if highest_form >= 4 and validate_bcnf(df, primary_keys, fds):
            output_file.write("BCNF validation successful.\n")
            current_nf = "BCNF"
        else:
            output_file.write("BCNF decomposition needed.\n")

        decomposed_tables = [(df, primary_keys)]
        if highest_form >= 4:
            decomposed_tables = apply_4nf(df, mvds)
            for i, (table, keys) in enumerate(decomposed_tables, start=1):
                table_name = f"Table_4NF_{i}"
                generate_sql_schema(table_name, table, keys)
            current_nf = 4 if current_nf != "BCNF" else current_nf

        if highest_form == 5:
            decomposed_tables = apply_5nf(df, jds)
            for i, (table, keys) in enumerate(decomposed_tables, start=1):
                table_name = f"Table_5NF_{i}"
                generate_sql_schema(table_name, table, keys)

        output_file.write(f"The highest normal form achieved: {current_nf}\n")

    # Execute the normalization
    normalize_data(data, primary_keys, functional_dependencies, multi_valued_dependencies, highest_form, join_dependencies)

print(f"Normalization results saved to {output_file_path}")


Enter primary key(s), separated by commas: OrderID, CustomerID, DrinkID, FoodID
Provide Functional Dependencies in the format 'A,B -> C,D' (type 'done' when finished):
OrderID -> CustomerID
done
Select the highest normal form to achieve (1: 1NF, 2: 2NF, 3: 3NF, B: BCNF, 4: 4NF, 5: 5NF): 4
Provide Multi-Valued Dependencies in the format 'A ->> B' (type 'done' when finished):
OrderID ->> DrinkID
done
CREATE TABLE Table_1NF (
  OrderID INT,
  Date VARCHAR(255),
  PromocodeUsed VARCHAR(255),
  TotalCost VARCHAR(255),
  TotalDrinkCost VARCHAR(255),
  TotalFoodCost VARCHAR(255),
  CustomerID INT,
  CustomerName VARCHAR(255),
  DrinkID INT,
  DrinkName VARCHAR(255),
  DrinkSize VARCHAR(255),
  DrinkQuantity INT,
  Milk VARCHAR(255),
  DrinkIngredient VARCHAR(255),
  DrinkAllergen VARCHAR(255),
  FoodID INT,
  FoodName VARCHAR(255),
  FoodQuantity INT,
  FoodIngredient VARCHAR(255),
  FoodAllergen VARCHAR(255),
  PRIMARY KEY (OrderID, CustomerID, DrinkID, FoodID)
);
CREATE TABLE Table_4NF_1 (
